In [1]:
########
## Env Setup / Preamble
########
%reload_ext autoreload
%autoreload complete 

%reload_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

𝛑   = NamedExpression('𝛑', scipy.constants.pi)         # pi
𝛆_0 = NamedExpression('𝛆_0', scipy.constants.epsilon_0) # electric permittivity
𝛍_0 = NamedExpression('𝛍_0', scipy.constants.mu_0)     # magnetic permeability of vacuum

𝛋 = 1. / (4 * 𝛑 * 𝛆_0)      # Coulomb's constant in [ N·m²/C² ]
𝙦_e = NamedExpression('𝙦_e', -scipy.constants.elementary_charge)

𝒉_planck = NamedExpression('𝒉', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

𝛂 = NamedExpression('𝛂', scipy.constants.fine_structure)

𝒎_e = NamedExpression('𝒎_e', scipy.constants.electron_mass)
𝒎_p = NamedExpression('𝒎_p', scipy.constants.proton_mass)

amu = NamedExpression(
    'amu', 
    scipy.constants.physical_constants['atomic mass constant'][0]
)

################

#Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

# [7]

do a rotation on a 2D metric (like the circle/polar problem) and show same answer

# [14]

Figure out the metric for a 2D curved earth with longitude/latitude coordinates.

In [2]:
var("R")
S2 = manifolds.Sphere(2, radius=R)
R3 = S2.ambient()

spherical.<𝛉, 𝛗> = S2.spherical_coordinates()
spherical


Chart (A, (theta, phi))

In [ ]:
# South Pole, 𝛉 = 𝛑
spherical.coord_range()


theta: (0, pi); phi: [-pi, pi] (periodic)

In [4]:
cartesian.<x,y,z> = R3.chart()

cartesian


Chart (E^3, (x, y, z))

In [5]:
𝚽 = S2.embedding()
𝚽.display()


iota: S^2_R → E^3
on A: (theta, phi) ↦ (x, y, z) = (R*cos(phi)*sin(theta), R*sin(phi)*sin(theta), R*cos(theta))

In [6]:
g = S2.induced_metric()
g[:]


[             R^2                0]
[               0 R^2*sin(theta)^2]

In [7]:
g.display()


g = R^2 dtheta⊗dtheta + R^2*sin(theta)^2 dphi⊗dphi

In [8]:
U2 = S2.open_subset('U')

# South Pole, West - Negative Sign 
# North Pole, East - Positive Sign
latlong.<𝛌,l> = U2.chart(r'𝛌:[-180,180] l:[-90,90]')

latlong.transition_map(
    spherical,
    [
        (𝛑/2) + ((-𝛑 / 180.) * l),     # latitude to φ
        (𝛑 / 180.) * 𝛌,                 # longitude to θ
    ]
)



Change of coordinates from Chart (A_inter_U, (𝛌, l)) to Chart (A_inter_U, (theta, phi))

# [15]

Find EoM for and solve it for Denver to London

In [9]:
denver = S2.point( (-104.9847, 39.73915), chart=latlong, name="Denver")
print(denver)
print(latlong(denver))
spherical(denver)


Point Denver on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3
(-104.984700000000, 39.7391500000000)


(0.8772173173509918, -1.832328679218491)

In [10]:
london = S2.point( (-0.12574, 51.50853), chart=latlong, name="London")
print(london)
print(latlong(london))
spherical(london)


Point London on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3
(-0.125740000000000, 51.5085300000000)


(0.6718028854326217, -0.00219457700145767)

# [16]

Calculate the Reimann curvature tensor for a flat space in polar coordinates and on the sphere S2.

# [17]

Compute the change in a vector due to transport around close curve in S2

# [19]

You watch a muon speed to earth with energy 100 GeV. How long will it take to decay?  To reach the ground from the upper atmosphere?

# [20]

show that the momentum 4-vector has an invariant length of -m^2c^4

# [22]

Get the Ricci Tensor/scalar to show the field equation for huv is correct

# [23]

Get hmn for sun and show that it satisfies the linearized equation, get geodesic equation for light from hmn

# [24] 

Reproduce figure 4.5 from pg 140 of the text on the deflection of light.

# [25]

verify the solution of the scalar wave equation, the "plane wave" solution.

# [26]

Do Exercises 3, 4, and 5 in Section 3.1 of Ohanian.


## §3.1 ex. 3

>   Show that the gauge transformation (3.19) produces no change in $F_{\mu\nu}$.
>
>   $$ A^\nu \rightarrow A^\nu + \partial^\nu \Lambda \tag{3.19} $$
>
>   given: 
>
>   *   $$ F^{\mu\nu} \equiv \partial^\mu A^\nu - \partial^\nu A^\mu \tag{3.15} $$
>   *   $\Lambda$ is any arbitrary scalar function, the _gauge function_.  

## §3.1 ex. 4

>   Show that for a wave propagating in the $z$-direction, Eqs. (3.24) and (3.20), 
>   with $j^\nu = 0$, have the following linearly independent solutions:
>
>   $$ A^\nu = \epsilon_{(n)}^\nu \cos\left(\omega z - \omega t\right) \tag{3.26}$$
>
>   where $\epsilon_{(n)}^{\nu} : n \in \{1, 2, 3\}$ is one of the following vectors:
>
>   * $\epsilon_{(1)}^\nu = (0, 1, 0, 0)$
>   * $\epsilon_{(2)}^\nu = (0, 0, 1, 0)$
>   * $\epsilon_{(3)}^\nu = (1, 0, 0, 1)$
>
>   (Note :> The subscript $n$ has been enclosed in parentheses to indicate that it is 
>    not a tensor index; it is only a label that distinguishes between the four&ndash;
>    vectors above.)
>
>   Show that any other solution of the type $\cos\left(\omega z - \omega t\right)$
>   can be written as a linear combination of the three solutions, with the 
>   _polarization vectors_ $\epsilon_{(n)}^\nu$ listed above. 

## §3.1 ex. 5

>   Find the $\mathbf{E}$ and $\mathbf{B}$ fields for each of the solutions 
>   $\epsilon_{(n)}^\nu$, and show that the $\epsilon_{(3)}^\nu$ solution 
>   gives $\mathbf{E} = \mathbf{B} = 0$.
>
>   Hence, only the solutions with the first two polarization vectors $\epsilon_{(1)}^\nu$
>   and $\epsilon_{(2)}^\nu$ have any physical significance. 
>
>   These two polarization vectors correspond to the familiar two independent, 
>   perpendicular directions of polarization of an electromagnetic wave.  Show that
>   the solution with the polarization vector $\epsilon_{(3)}$\nu$ can be
>   completely eliminated by a gauge transformation, with $\Lambda = (1/\omega)
>   \sin\left(\omega z - \omega t\right)$, which leaves to $\overline{A}^\nu = 0$. 

# [27]

Compute the effect of a grav. wave traveling in the z-dir on a circle of masses arranged in the x-y plane.